### Mount Google Drive

Mount your Google Drive on your runtime using an authorization code.

Follow the instruction on the screen to authorise Colab accessing your drive. On your Google Drive, you can access you files prepend with “/content/drive/MyDrive/”

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change working director
%cd "/content/drive/MyDrive/Colab Notebooks/goML"

### Import necessary libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from seaborn import load_dataset


In [ ]:
#Load our data set
df = pd.read_csv("data/titanic.csv")

In [ ]:
df

In [ ]:
df.info()

As you can see we have null values in ‘Age’ and 'Cabin' columns. We will drop some of them, and deal with the rest.

In [ ]:
columns = ['Cabin']
df_2 = df.drop(columns, axis=1)

‘columns’ list contains the names which I want to drop from my dataset. ‘drop’ method will drop them. axis=1, means we want as column to drop.

Name, Ticket and PassengerID are also not going to be useful for us.  We wil drop this column as well.


In [ ]:
columns = ['Name','Ticket', 'PassengerId']
df_3 = df_2.drop(columns, axis=1)

In [ ]:
df_3.describe(include='all').T

In [ ]:
print(f"Max value of age column : {df_3['Age'].max()}")
print(f"Min value of age column : {df_3['Age'].min()}")

We can categorize age column with values between 0 and 80.

Pandas ‘cut’ method will let us make our own categorization.

In [ ]:
bins = [0, 5, 17, 25, 50, 80]
labels = ['Infant', 'Kid', 'Young', 'Adult', 'Old']
df_3['Age'] = pd.cut(df_3['Age'], bins = bins, labels=labels)

In [ ]:
pd.DataFrame(df_3['Age'].value_counts())

We can see that adults are in the majority.  We still have null values in ‘age’ column.  Let's have a look at the mode of Age column.

In [ ]:
df_3['Age'].mode()[0]

We can fill null values with mode of this column. This is an option.

In [ ]:
df_4 = df_3.fillna({'Age' : df_3['Age'].mode()[0]})

We are done with ‘age’ column. Yes! It’s time for ‘Embarked’ column.

In [ ]:
df_4['Embarked'].unique()

In [ ]:
print(f"How many 'S' on embarked column : {df_3[df_3['Embarked'] == 'S'].shape[0]}")
print(f"How many 'C' on embarked column : {df_3[df_3['Embarked'] == 'C'].shape[0]}")
print(f"How many 'Q' on embarked column : {df_3[df_3['Embarked'] == 'Q'].shape[0]}")

Looks like we can use mode of this column to fill nan values.

In [ ]:
df_4 = df_4.fillna({'Embarked' : 'S'})

In [ ]:
df_4['Embarked'].unique()

In [ ]:
print(f"How many 'S' on embarked column : {df_4[df_4['Embarked'] == 'S'].shape[0]}")
print(f"How many 'C' on embarked column : {df_4[df_4['Embarked'] == 'C'].shape[0]}")
print(f"How many 'Q' on embarked column : {df_4[df_4['Embarked'] == 'Q'].shape[0]}")

In [ ]:
df_4[['Pclass', 'Survived']].groupby(['Pclass']).sum().sort_values(by='Survived')

In [ ]:
df_4[['Sex', 'Survived']].groupby(['Sex']).sum().sort_values(by='Survived')

We can categorize fare column too, like the code above.

In [ ]:
bins = [-1, 7.9104, 14.4542, 31, 512.330]
labels = ['low', 'medium-low', 'medium', 'high']
df_4['Fare'] = pd.cut(df_4['Fare'], bins = bins, labels = labels)

Let's take a look at our final version of our dataset

In [ ]:
df_4

In [ ]:
sns.distplot(df_4['Survived'])

Now, machine learning models hates non-numeric values. We cannot put them to our train and test data. We need to convert them into numeric values. You have two options for that; Label Encoder, and Pandas get_dummies method (One-Hot Encoding).  We will use get_dummies. One-hot encoding is more suitable for machine learning. Because labels are independent to each other, e.g. 2 doesn't mean twice that value of 1.

In [ ]:
dummies = ['Fare', 'Age', 'Embarked', 'Sex']
dummy_data = pd.get_dummies(df_4[dummies])

‘dummies’ contains column names that we want to convert into numeric values. Every variable in them will become a column, and their absence will define as 0, and 1 whether they’re true for that passenger or not.

In [ ]:
dummy_data.shape

In [ ]:
dummy_data

We will concat the two data frames, and drop the old columns.

In [ ]:
df_5 = pd.concat([df_4, dummy_data], axis = 1)
df_5.drop(dummies, axis=1, inplace=True)

Now we have 891 rows, 18 columns. We are ready to build our model.

In [ ]:
df_5.shape

### Split dataset

X contains independent values, y contains dependent value

In [ ]:
X = df_5.drop('Survived', axis = 1)
y = df_5['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

Model Building

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
y_pred

We can check the accuracy score of our model.

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_pred, y_test)

In [ ]:
metrics.confusion_matrix(y_pred, y_test)

In [ ]:
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

Let's try out other classification algorithms.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gaussianNB = GaussianNB()
gaussianNB.fit(X_train, y_train)
y_pred = gaussianNB.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
cart = DecisionTreeClassifier()
cart.fit(X_train, y_train)
y_pred = cart.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
metrics.accuracy_score(y_pred, y_test)

### Save model

In [ ]:
import joblib
# Save the trained model to a file so we can use it to make predictions later
joblib.dump(xgb, 'models/titanic_model.pkl')

### Load model

In [ ]:
# Load our trained model
model = joblib.load('models/titanic_model.pkl')

### Make Inference

In [ ]:
# Define the house that we want to value (with the values in the same order as in the training data)
passenger_1 = [
    1,  # Pclass
    1,  # SibSp
    0,  # Parch
    1,  # Fare_low
    0,  # Fare_medium-low
    0,  # Fare_medium
    0,  # Fare_high
    0,  # Age_Infant
    0,  # Age_Kid
    0,  # Age_Young
    1,  # Age_Adult
    0,  # Age_Old
    1,  # Embarked_C
    0,  # Embarked_Q
    0,  # Embarked_S
    0,  # Sex_female
    1,  # Sex_male
]


In [ ]:
# scikit-learn assumes you want to predict the values for multiple of passengers at once, so it expects an array.
# We only want to estimate the value of a single passenger, so there will only be one item in our array.
columns = ['Pclass','SibSp','Parch','Fare_low','Fare_medium-low','Fare_medium','Fare_high','Age_Infant'
,'Age_Kid','Age_Young','Age_Adult','Age_Old','Embarked_C','Embarked_Q','Embarked_S','Sex_female','Sex_male']
passengers = pd.DataFrame([passenger_1],columns=columns)

In [ ]:
# Make a prediction for each house in the homes array (we only have one)
prep_values = model.predict(passengers)

# Since we are only predicting the price of one house, grab the first prediction returned
predicted_value = prep_values[0]

# Print the results
print("Predicated class: %s"%predicted_value)